In [3]:
from langchain.document_loaders import PyPDFDirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Pinecone
import pinecone
from transformers import AutoTokenizer, AutoModelForCausalLM
from transformers import pipeline
import os
import sys
from langchain import HuggingFacePipeline, PromptTemplate
from langchain.chains import RetrievalQA
import tiktoken
import PyPDF2
from langchain.document_loaders import UnstructuredPDFLoader, OnlinePDFLoader, PyPDFLoader
from langchain.vectorstores import Chroma
import json
import datetime
from torch import cuda
import torch
import pandas as pd

In [59]:
import requests
import os


import requests
import base64
from urllib.parse import urlparse, parse_qs

def extract_pdf_url(url):
    """
    Extracts the actual PDF URL from the given URL.
    Decodes it from base64 if necessary.
    """
    if url.lower().endswith('.pdf'):
        return url  # Direct PDF URL
    else:
        parsed_url = urlparse(url)
        query_params = parse_qs(parsed_url.query)
        pdf_target = query_params.get('pdfTarget', [None])[0]

        if pdf_target:
            pdf_url = base64.b64decode(pdf_target).decode('utf-8')
            return pdf_url
        else:
            raise ValueError("No valid PDF URL found in the provided URL")

def download_pdf(url, save_path):
    """
    Downloads a PDF from a given URL.
    """
    try:
        pdf_url = extract_pdf_url(url)
        response = requests.get(pdf_url, stream=True)
        response.raise_for_status()  # Ensure the request was successful

        with open(save_path, 'wb') as file:
            for chunk in response.iter_content(chunk_size=8192):
                file.write(chunk)

        print(f"Downloaded PDF from: {pdf_url} to {save_path}")
    except Exception as e:
        print(f"Error downloading PDF: {e}")


# Example usage



def download_all_pdfs(dataset):
    """
    Download all PDFs in the given dataset.
    """
    for row in dataset:
        doc_name = row['doc_name']
        doc_link = row['doc_link']
        if doc_link:  # Check if the link is not empty
            # Create a directory for downloads if it doesn't exist
            download_dir = "downloads"
            os.makedirs(download_dir, exist_ok=True)
            # Define the path where the PDF will be saved
            save_path = os.path.join(download_dir, f"{doc_name}.pdf")
            download_pdf(doc_link, save_path)

# Call the function to download all PDFs
#download_all_pdfs(dataset['train'])  # Assuming you want to download from the 'train' split


In [5]:
from huggingface_hub import notebook_login
notebook_login()

In [19]:

def create_knowledge_hub(path_to_10k, doc_name):
    """From a 10-K document, create or use an existing Chroma DB knowledge hub.

    Args:
        path_to_10k: Relative path to the 10-K hosted locally on the user's computer
        doc_name: The name of the document, used to identify the vector database

    Returns:
        vectordb: The vector database with the information from the 10-K
        db_directory: The path to the vector database
    """

    # Normalize doc_name to create a valid directory name
    normalized_doc_name = doc_name.replace(' ', '_').replace('/', '_')
    
    # Define the base directory for vector databases
    base_dir = "vector_db"
    # Ensure the base directory exists
    if not os.path.exists(base_dir):
        os.makedirs(base_dir)

    # Define the full path to the specific database directory
    db_directory = os.path.join(base_dir, "db_" + normalized_doc_name)
    
    embed_model_id = 'sentence-transformers/all-MiniLM-L6-v2'

    device = f'cuda:{cuda.current_device()}' if cuda.is_available() else 'cpu'

    embed_model = HuggingFaceEmbeddings(
        model_name=embed_model_id,
        model_kwargs={'device': device},
        encode_kwargs={'device': device, 'batch_size': 32}
    )

    # Check if the database directory already exists
    if os.path.exists(db_directory):
        print(f"Using existing database for document: {doc_name}")
        vectordb = Chroma(persist_directory=db_directory, embedding_function=embed_model)
    else:
        print(f"Creating new database for document: {doc_name}")

        loader = PyPDFLoader(path_to_10k)
        documents = loader.load()

        splitter = RecursiveCharacterTextSplitter(
            chunk_size=1300, 
            chunk_overlap=5,
            separators=["\n\n", "\n", " ", ""],
            length_function=len)
        texts = splitter.split_documents(documents)

        vectordb = Chroma.from_documents(
            documents=texts, 
            embedding=embed_model,
            persist_directory=db_directory
        )
        vectordb.persist()

    return vectordb, db_directory


In [20]:
def query_model(path_to_10k, doc_name, question):
    """Asks LLAMA a question based off a local 10-K document.

    Args:
        path_to_10k: Relative path to the 10-K hosted locally on the user's computer
        question: Question to ask the model

    Returns:
        answer: The answer given by the fine-tuned GPT model
    """

    db, db_dir = create_knowledge_hub(path_to_10k, doc_name)

    source1 = db.similarity_search(question, k = 2)[0].page_content
    source2 = db.similarity_search(question, k = 2)[1].page_content

    ## EDIT THIS PART
    model = AutoModelForCausalLM.from_pretrained("meta-llama/Llama-2-7b-chat-hf",
                                             device_map='auto',
                                             torch_dtype=torch.float32,
                                             use_auth_token=True,
                                             load_in_8bit=False
                                          )
    
    tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-2-7b-chat-hf", use_auth_token=True)
    
    retriever = db.as_retriever(search_kwargs={"k": 2})

    pipe = pipeline("text-generation",
                model=model,
                tokenizer= tokenizer,
                torch_dtype=torch.float,
                device_map="auto",
                max_new_tokens = 512,
                do_sample=True,
                top_k=30,
                num_return_sequences=1,
                eos_token_id=tokenizer.eos_token_id
                )
    
    llm=HuggingFacePipeline(pipeline=pipe, model_kwargs={'temperature':0.1})
    
    qa_chain = RetrievalQA.from_chain_type(llm=llm,
                                  chain_type="stuff",
                                  retriever=retriever,
                                  return_source_documents=True)
    ## END OF EDITING

    #delete_chroma_db(db_dir)
    
    answer = qa_chain(question)['result']
    

    return answer

In [21]:
import sklearn
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

def compare_strings(text1, text2):
    vectorizer = TfidfVectorizer()
    vectors = vectorizer.fit_transform([text1, text2])
    # Calculate the cosine similarity between the vectors
    similarity = cosine_similarity(vectors)
    return similarity[0][1]

In [22]:
import nltk
from nltk.util import ngrams
from nltk.metrics import edit_distance

def compute_rouge_l(candidate, reference):
    m, n = len(candidate), len(reference)
    print(m,n)
    dp_table = [[0] * (n + 1) for _ in range(m + 1)]
    
    for i in range(1, m + 1):
        for j in range(1, n + 1):
            if candidate[i - 1] == reference[j - 1]:
                dp_table[i][j] = dp_table[i - 1][j - 1] + 1
            else:
                dp_table[i][j] = max(dp_table[i - 1][j], dp_table[i][j - 1])
    
    return dp_table[m][n] / n

In [23]:
#path_to_csv_dataset = "./financebench/financebench_sample_150.csv" 
path_to_csv_dataset = "train.csv"

In [60]:
import pandas as pd
import os

def run_eval():
    # Initialize lists to store values for each column
    answers = []
    model_answers = []
    cosine_similarities = []
    rouge_l_scores = []

    dataset = load_dataset("PatronusAI/financebench")
    df = dataset['train'].to_pandas()[:50]

    for index, row in df.iterrows():
        question = row['question']
        answer = row['answer']
        doc_name = row['doc_name']
        doc_link = row['doc_link']
        
        download_dir = "documents"
        os.makedirs(download_dir, exist_ok=True)
        save_path = os.path.join(download_dir, f"{doc_name}.pdf")
        
        download_pdf(doc_link, save_path)
        
        model_answer = query_model(save_path, doc_name, question)
        
        if isinstance(model_answer, dict):
            model_answer = model_answer.get("key_for_answer", "")

        sim = compare_strings(answer, model_answer)
        rouge_l = compute_rouge_l(model_answer, answer)

        # Store the values in their respective lists
        answers.append(answer)
        model_answers.append(model_answer)
        cosine_similarities.append(sim)
        rouge_l_scores.append(rouge_l)
        
        print("Refrence Answer: ", answer) 
        print("model Answer: ", model_answer)
        print("Cosine Similarity is: ", sim )
        print("Rouge-L score is: ", rouge_l_scores)


        # Delete the document downloaded if necessary
        # delete_document(filename)

    # Create a DataFrame from the lists
    result_df = pd.DataFrame({
        'answer': answers,
        'model_answer': model_answers,
        'cosine_similarity': cosine_similarities,
        'rouge_l_score': rouge_l_scores
    })

    # Optionally, return the average similarity along with the DataFrame
    #average_similarity = sum(cosine_similarities) / len(cosine_similarities)
    
    return result_df

# Ensure to define or import the functions download_pdf, query_model, and compare_strings as they are used in this script.


In [58]:
results = run_eval()


Downloaded PDF from: https://www.adobe.com/content/dam/cc/en/investor-relations/pdfs/ADBE-10K-FY15-FINAL.pdf to documents\ADOBE_2015_10K.pdf
Creating new database for document: ADOBE_2015_10K


C:\Users\spurt\.conda\envs\Anote\lib\site-packages\transformers\models\auto\auto_factory.py:472: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

C:\Users\spurt\.conda\envs\Anote\lib\site-packages\transformers\models\auto\tokenization_auto.py:690: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


526 4
Refrence Answer:  0.66
model Answer:   I don't know the answer to your question because the information provided in the given statements does not contain enough data to calculate the FY2015 operating cash flow ratio for Adobe. The statements only provide information on the company's cash and cash equivalents, net increase (decrease) in cash and cash equivalents, and cash flows from operating activities, but do not provide the total current liabilities required to calculate the operating cash flow ratio. Therefore, I cannot provide an answer to your question.
Cosine Similarity is:  0.0
Rouge-L score is:  [0.5]


In [61]:
results_df = run_eval()

Downloaded PDF from: https://investors.3m.com/financials/sec-filings/content/0001558370-19-000470/0001558370-19-000470.pdf to documents\3M_2018_10K.pdf
Using existing database for document: 3M_2018_10K


C:\Users\spurt\.conda\envs\Anote\lib\site-packages\transformers\models\auto\auto_factory.py:472: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

C:\Users\spurt\.conda\envs\Anote\lib\site-packages\transformers\models\auto\tokenization_auto.py:690: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


152 8
Refrence Answer:  $1577.00
model Answer:   The FY2018 capital expenditure amount for 3M is not directly disclosed in the cash flow statement. Therefore, I don't know the answer to this question.
Cosine Similarity is:  0.0
Rouge-L score is:  [0.25]
Downloaded PDF from: https://investors.3m.com/financials/sec-filings/content/0001558370-19-000470/0001558370-19-000470.pdf to documents\3M_2018_10K.pdf
Using existing database for document: 3M_2018_10K


...

135 4
Refrence Answer:  2.8%
model Answer:   Based on the information provided in the income statement, the FY2015 - FY2017 3 year average net profit margin for Best Buy is 3.57%.
Cosine Similarity is:  0.0
Rouge-L score is:  [0.25, 0.4, 0.5561224489795918, 0.37966101694915255, 0.6041666666666666, 0.7064220183486238, 0.39591836734693875, 0.7553956834532374, 0.6, 0.75, 0.5, 0.4, 0.75, 0.8467741935483871, 0.6428571428571429, 1.0, 0.8775510204081632, 0.8, 0.8, 1.0, 0.6666666666666666, 1.0, 0.3118279569892473, 0.8411214953271028, 0.4940828402366864, 0.8028169014084507, 0.8028169014084507, 0.8082191780821918, 0.5813953488372093, 0.9574468085106383, 0.75, 0.7954545454545454, 0.5687103594080338, 0.5460526315789473, 0.9333333333333333, 0.8070175438596491, 0.8181818181818182, 0.8412698412698413, 1.0, 0.8571428571428571, 0.9807692307692307, 1.0, 0.8679245283018868, 0.9411764705882353, 1.0, 0.4, 0.75, 0.7402597402597403, 0.75]
Downloaded PDF from: https://d18rn0p25nwr6d.cloudfront.net/CIK-0000764

C:\Users\spurt\.conda\envs\Anote\lib\site-packages\transformers\models\auto\auto_factory.py:472: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

C:\Users\spurt\.conda\envs\Anote\lib\site-packages\transformers\models\auto\tokenization_auto.py:690: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


188 8
Refrence Answer:  $5409.00
model Answer:   Based on the information provided in the balance sheet of Best Buy's annual report for FY2019, the total amount of inventories as of the end of FY2019 was approximately USD 3,425 million.
Cosine Similarity is:  0.0
Rouge-L score is:  [0.25, 0.4, 0.5561224489795918, 0.37966101694915255, 0.6041666666666666, 0.7064220183486238, 0.39591836734693875, 0.7553956834532374, 0.6, 0.75, 0.5, 0.4, 0.75, 0.8467741935483871, 0.6428571428571429, 1.0, 0.8775510204081632, 0.8, 0.8, 1.0, 0.6666666666666666, 1.0, 0.3118279569892473, 0.8411214953271028, 0.4940828402366864, 0.8028169014084507, 0.8028169014084507, 0.8082191780821918, 0.5813953488372093, 0.9574468085106383, 0.75, 0.7954545454545454, 0.5687103594080338, 0.5460526315789473, 0.9333333333333333, 0.8070175438596491, 0.8181818181818182, 0.8412698412698413, 1.0, 0.8571428571428571, 0.9807692307692307, 1.0, 0.8679245283018868, 0.9411764705882353, 1.0, 0.4, 0.75, 0.7402597402597403, 0.75, 0.375]


In [62]:
results_df

,answer,model_answer,cosine_similarity,rouge_l_score
0,$1577.00,The FY2018 capital expenditure amount for 3M ...,0.000000,0.250000
1,$8.70,\nThe net PPNE for 3M at the end of FY2018 is ...,0.000000,0.400000
2,"No, the company is managing its CAPEX and Fixe...","Yes or No, with an explanation if you have on...",0.097621,0.556122
3,Operating Margin for 3M in FY2022 has decrease...,\nIf you don't know the answer to this questio...,0.126768,0.379661
4,The consumer segment shrunk by 0.9% organically.,3M does not provide segment information exclu...,0.091940,0.604167
5,No. The quick ratio for 3M was 0.96 by Jun'23 ...,The quick ratio is a commonly used metric to ...,0.227010,0.706422
6,Following debt securities registered under 3M'...,Based on the information provided in the text...,0.122194,0.395918
7,"Yes, not only they distribute the dividends on...",3M has paid dividends since 1916 and has a hi...,0.190924,0.755396
8,24.26,We are unable to calculate the fixed asset tu...,0.000000,0.600000
9,1.9%,Capex as a percentage of revenue is calculate...,0.000000,0.750000


In [63]:
results_df.to_csv("Rag_results.csv", index = False)